# Access Data4Human Sentinel-1 Floodmasks

In [ ]:
import dask.diagnostics
import folium
import folium.plugins as plugins
import geopandas as gpd

from odc.stac import load
from pystac_client import Client
from shapely.geometry import shape

import pystac

## Query STAC API
The easiest way to work with the data is to use the [Geoservice STAC API](https://geoservice.dlr.de/eoc/ogc/stac). The Data4Human Sentinel-1 Floodmasks are located in the [D4H collection](https://geoservice.dlr.de/eoc/ogc/stac/collections/D4H). In the following we use [pystac-client](https://github.com/stac-utils/pystac-client) to query the items in this collections by bounding box (*bbox*) and acquisition dates (*dates_from_to*). Optionally, we can also filter the items by properties such as when they have been updated.

In [ ]:
# search stac api
stacapi_endpoint = "https://geoservice.dlr.de/eoc/ogc/stac/v1"
collections = ["D4H"]
bbox = [33.851, -20.63, 35.859, -18.88]
dates_from_to = ["2019-03-01", "2019-03-31"]
filt = None #"updated > '2022-02-20T00:00:00.000' AND updated < '2022-02-28T10:00:00.000'"

catalog = Client.open(
    url=stacapi_endpoint,
    ignore_conformance=True
)

search = catalog.search(
    collections=collections, 
    bbox=bbox, 
    datetime=dates_from_to, 
    filter=filt, 
    method="GET", 
    filter_lang="cql2-text",
    max_items=3000
)

items = [item for item in search.items()]
print(f"Search returned {len(items)} items")

## Convert items to Geopandas Dataframe
We can convert the pystac items to a Geopandas Dataframe for further vector analysis or simply to save them to file in a range of GIS file formats.

In [ ]:
items_dict = [item.to_dict() for item in items]
items_gdf = gpd.GeoDataFrame.from_features(items_dict)
items_gdf.head(3)

## Visualize item footprints on map
We can use the Geopandas Dataframe to easily visualize the item footprints on an interactive Leaflet (Folium) map.

In [ ]:
m = folium.Map(tiles="Stamen Terrain")
layer_control = folium.LayerControl(position="topright", collapsed=True)
fullscreen = folium.plugins.Fullscreen()
marker_cluster = folium.plugins.MarkerCluster(name="Item markers").add_to(m)

for item in items:
    geom_pt = shape(item.geometry).representative_point()
    folium.Marker(
        [geom_pt.y, geom_pt.x], 
        popup=f"<a href={item.links[4].target} target='_blank' rel='noopener noreferrer'>{item.id}</a><br>"
        f"platform: {item.properties['platform']}<br>"
        f"datetime: {item.datetime}"
    ).add_to(marker_cluster)
    
style = {"fillColor": '#00000000', "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    items_gdf.to_json(), 
    name="Item footprints", 
    style_function=lambda x: style,
    control=True
)

footprints.add_to(m)
layer_control.add_to(m)
fullscreen.add_to(m)
m.fit_bounds(m.get_bounds())
m 

## Convert items to Xarray Dataset
We can convert the pystac items to a Xarray Dataset for further raster analysis, such as timeseries analysis or mosaicing. Note that here we set a lower resolution for faster visualization (the native resolution of the products is 0.0001).

In [ ]:
items_ds = load(
    items,
    bands=["WATER", "VALID"],
    crs="EPSG:4326",
    resolution=0.001,
    lon=(bbox[0], bbox[2]),
    lat=(bbox[1], bbox[3]),
    skip_broken_datasets=True,
    chunks={"latitude": 2048, "longitude": 2048},
)  
print(items_ds)

## Compute coverage (number of valid observations) from valid mask assets
Each item has several assets assigned to it. For the Data4Human Sentinel-1 Floodmasks the assets are:

- "VALID": Valid pixel mask
- "WATER": Water mask
- "REFER": Reference water mask

In the following we use the valid pixel mask assets to compute the number of valid observations per pixel, which provides us with an efficient way to get the satellite coverage for the given area and time of interest.

In [ ]:
# sum valid masks along time dimension to compute number of valid observations
coverage = items_ds["VALID"].sum(dim="time")

# compute and plot coveragemask
with dask.diagnostics.ProgressBar():
    data = coverage.compute()
    data.plot(size=10, cbar_kwargs={'label': "Coverage (valid observations)"})

## Create mosaic from water mask assets
Here we get the maximum value along time dimension to mosaic all water masks assets.

In [ ]:
# get maximum value along time dimension to mosaic water masks
mosaic = items_ds["WATER"].max(dim="time")

# compute and plot mosaic
with dask.diagnostics.ProgressBar():
    data = mosaic.compute()
    data.plot(size=10, cmap="Blues", add_colorbar=False)

## Create water frequency and flood duration from water mask assets
Here we do some more time-series analysis to compute a simple water frequency layer and a more advanced flood duration layer.

In [ ]:
# sum water masks along time axis and divide by the number of valid observations
frequency = items_ds["WATER"].sum(dim="time") / items_ds["VALID"].sum(dim="time")

# compute and plot frequency
with dask.diagnostics.ProgressBar():
    data = frequency.compute()
    data.plot(size=10, cmap="Blues", vmin=0, vmax=1, cbar_kwargs={'label': "Relative water frequency"})

In [ ]:
# combine water and valid masks
# forward fill water masks where no valid observations are available
# set remaining nan values (at first timestamp) to 0
water_ds = items_ds["WATER"].where(items_ds["VALID"] == 1).ffill(dim="time").fillna(0)

# resample water masks to daily temporal resolution using forward fill and sum up
# this effectively provides us with the total flood duration in the requested reference time period
duration = water_ds.resample(time="1D").ffill().sum(dim="time")

# compute and plot duration
with dask.diagnostics.ProgressBar():
    data = duration.compute()
    data.plot(size=10, cmap="Blues", cbar_kwargs={'label': "Flood duration (days)"})

## Visualize water mask from WMS
Besides the STAC API endpoint, we can also use common OGC webservices such as Web Map Service (WMS) to interact with the products on Geoservice. In the following we simply load a WMS layer of the water mask for a desired timestamp. Please note that this examples requires the exact timestamp of a product. There are, however, more convenient tools and Leaflet Plugins available that allow to specify dates or date ranges instead.

In [ ]:
# NOTE: this requires the exact timestamp and accepts a list of timestamps seems
m = folium.Map(tiles="Stamen Terrain")
watermask = folium.raster_layers.WmsTileLayer(
    url = "https://geoservice.dlr.de/eoc/demo/wms?", 
    layers="D4H_WATER_MASK_S1", 
    time='2022-02-21T03:02:11Z', #f"{dates_from_to[0]}T00:00:00Z",
    #bounds=[[bbox[1], bbox[0]], [bbox[3], bbox[2]]],
    fmt='image/png', 
    transparent=True, 
    version='1.1.1',
    name="Water mask",
    control=True
)
watermask.add_to(m)
fullscreen.add_to(m)
m.fit_bounds(footprints.get_bounds())
m